In [31]:
import numpy as np

In [42]:
import importlib
import dataImportFuctions
import functions

importlib.reload(dataImportFuctions)
from dataImportFuctions import main_data_dic_generator
from dataImportFuctions import TS_extractor
from dataImportFuctions import TS_extractor_v2
from dataImportFuctions import TS_dic_to_NLDR_format

importlib.reload(functions)
from functions import estimate_global_dimension_pca
from functions import dictionary_random_reducer
from functions import mannwhitneyu_stat

## imprting all data of controls and patients for both parcellations

In [1]:
#for controls
controls_base_dir = '/Users/user/Int Dim NF1 WM/controls Schaeffer300_AAL3_timeseries_00301_Run1'
controls_file_pattern = 'Schaeffer300_AAL3_timeseries_{:03d}_Run1.mat'  # {:03d} will be replaced by numbers with leading zeros
#controls_index_list= [3,4,6,7,8,10,11,12,13,14,15,17,18,19]
controls_index_list= [1,2,3,4,6,7,8,10,11,12,13,14,15,17,18,19] #the original
N_c= len(controls_index_list)

#for patients
patients_base_dir = '/Users/user/Int Dim NF1 WM/patients Schaeffer300_AAL3_timeseries_00301_Run1'
patients_file_pattern = 'Schaeffer300_AAL3_timeseries_{:03d}01_Run1.mat'
#patients_index_list= [3,4,5,6,7,8,11,12,13,14,15,16,17,18,19,21,22,23,24,25,26,27,29,30,32,33]
patients_index_list= [1,2,3,4,5,6,7,8,11,12,13,14,15,16,17,18,19,21,22,23,24,25,26,27,29,30,32,33] #the original
N_p= len(patients_index_list)

In [6]:
#controls
#AAL3
controls_AAL3_main_dic= main_data_dic_generator(controls_base_dir,
                                           controls_file_pattern, 
                                           controls_index_list,
                                           'AAL3',
                                           False) #print_bool
#Schaeffer300                     
controls_Schaeffer300_main_dic= main_data_dic_generator(controls_base_dir,
                                           controls_file_pattern, 
                                           controls_index_list,
                                           'Schaeffer300',
                                           False) #print_bool

#patients
#AAL3
patients_AAL3_main_dic= main_data_dic_generator(patients_base_dir,
                                           patients_file_pattern, 
                                           patients_index_list,
                                           'AAL3',
                                           False) #print_bool
#Schaeffer300                                           
patients_Schaeffer300_main_dic= main_data_dic_generator(patients_base_dir,
                                           patients_file_pattern, 
                                           patients_index_list,
                                           'Schaeffer300',
                                           False) #print_bool                                    

In [ ]:
# print(controls_AAL3_main_dic.keys())
# print(controls_AAL3_main_dic['001'].shape)

dict_keys(['001', '002', '003', '004', '006', '007', '008', '010', '011', '012', '013', '014', '015', '017', '018', '019'])
(144, 166)


In [22]:
#controls
#AAL3
controls_AAL3_dic_2back_TSs, controls_AAL3_dic_0back_TSs, controls_AAL3_dic_entire_TSs = TS_extractor_v2(controls_AAL3_main_dic, 
                False) #print_bool
#Schaeffer300  
controls_Schaeffer300_dic_2back_TSs, controls_Schaeffer300_dic_0back_TSs, controls_Schaeffer300_dic_entire_TSs = TS_extractor_v2(controls_Schaeffer300_main_dic,
                False) #print_bool

#patients
#AAL3
patients_AAL3_dic_2back_TSs, patients_AAL3_dic_0back_TSs, patients_AAL3_dic_entire_TSs = TS_extractor_v2(patients_AAL3_main_dic,
                False) #print_bool
#Schaeffer300  
patients_Schaeffer300_dic_2back_TSs, patients_Schaeffer300_dic_0back_TSs, patients_Schaeffer300_dic_entire_TSs = TS_extractor_v2(patients_Schaeffer300_main_dic,
                False) #print_bool

In [23]:
print(controls_AAL3_dic_2back_TSs['001'].shape)
print(controls_AAL3_dic_0back_TSs['001'].shape)
print(controls_AAL3_dic_entire_TSs['001'].shape)

(72, 166)
(72, 166)
(144, 166)


## Global intrinsic dimension of cohort space space

In [40]:
#Schaeffer300 0-back G-PCA patient vs controls 
percentage_list= np.linspace(0.8,0.99,20)
print(percentage_list)
bootstrap_seed_list= [12,85,45,96,98,65,44,9,85,21,174,762,228,111,764,224,601,381,55,100]
N_PCA= 8 #10

Schaeffer300_pca_GID_controls_0b_mean= []
Schaeffer300_pca_GID_controls_0b_std= []

Schaeffer300_pca_GID_patients_0b_mean= []
Schaeffer300_pca_GID_patients_0b_std= []

p_value_Schaeffer300_0b_controls_vs_patients= []


for perc in percentage_list:
    print("PCA threshold= ", perc)
    #controls
    GDim_list_controls= []
    for seedd in bootstrap_seed_list:
        small_dic= dictionary_random_reducer(controls_Schaeffer300_dic_0back_TSs, seedd, N_PCA)
        selected_data_set= TS_dic_to_NLDR_format(small_dic,N_PCA)
        GDim= estimate_global_dimension_pca(selected_data_set,perc)
        #print(selected_data_set.shape)
        #print(GDim)
        GDim_list_controls.append(GDim)
    print(GDim_list_controls)
    GDim_list= np.array(GDim_list_controls)
    Schaeffer300_pca_GID_controls_0b_mean.append(np.mean(GDim_list_controls))
    Schaeffer300_pca_GID_controls_0b_std.append(np.std(GDim_list_controls))

    #patients
    GDim_list_patients= []
    for seedd in bootstrap_seed_list:
        small_dic= dictionary_random_reducer(patients_Schaeffer300_dic_0back_TSs, seedd, N_PCA)
        selected_data_set= TS_dic_to_NLDR_format(small_dic,N_PCA)
        GDim= estimate_global_dimension_pca(selected_data_set,perc)
        #print(selected_data_set.shape)
        #print(GDim)
        GDim_list_patients.append(GDim)
    
    print(GDim_list_patients)
    GDim_list_patients= np.array(GDim_list_patients)
    Schaeffer300_pca_GID_patients_0b_mean.append(np.mean(GDim_list_patients))
    Schaeffer300_pca_GID_patients_0b_std.append(np.std(GDim_list_patients))

        #t_stat, t_stat_p_value = scipy.stats.ttest_ind(GDim_1, GDim_2)
    MW_pvalue =mannwhitneyu_stat(GDim_list_controls, GDim_list_patients)
    p_value_Schaeffer300_0b_controls_vs_patients.append(MW_pvalue)
    # if MW_pvalue <= 0.05 :
    #     print("@@@@@@@@@@@@@")
        #print('threshold= ', perc, 'p-value= ', MW_pvalue)





[0.8  0.81 0.82 0.83 0.84 0.85 0.86 0.87 0.88 0.89 0.9  0.91 0.92 0.93
 0.94 0.95 0.96 0.97 0.98 0.99]
PCA threshold=  0.8
[33, 35, 32, 32, 35, 32, 36, 34, 35, 31, 31, 31, 33, 32, 31, 32, 34, 34, 32, 31]
[36, 36, 36, 35, 37, 38, 36, 39, 36, 37, 34, 38, 39, 36, 34, 34, 36, 37, 35, 36]


NameError: name 'mannwhitneyu_stat' is not defined

(1152, 300)
(2304, 300)


[44, 46, 49, 52, 55, 58, 62, 66, 70, 75, 80, 86, 92, 100, 109, 119, 131, 147, 168, 200]
[41, 43, 46, 49, 52, 55, 58, 62, 66, 71, 76, 82, 88, 96, 104, 114, 126, 142, 163, 195]
[46, 49, 52, 55, 58, 62, 66, 70, 75, 81, 86, 93, 101, 109, 119, 130, 144, 161, 184, 217]
[47, 50, 52, 55, 58, 62, 65, 69, 73, 78, 83, 89, 96, 103, 112, 122, 134, 150, 171, 203]
[45, 47, 50, 53, 56, 59, 63, 67, 71, 76, 81, 87, 93, 100, 109, 119, 131, 147, 167, 200]
[50, 53, 56, 59, 63, 66, 71, 75, 80, 85, 91, 98, 105, 113, 123, 134, 148, 166, 188, 221]
